In [119]:

import lightgbm as lg
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics import average_precision_score
import numpy as np
import wandb
from wandb.lightgbm import wandb_callback, log_summary

Error in callback <function _WandbInit._resume_backend at 0x7f82dcf01790> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

Error in callback <function _WandbInit._pause_backend at 0x7f82dcf01550> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

1. Load data

In [ ]:
train_df = pd.read_csv('/Users/favea/Downloads/swiss-data/train_df_balanced_0.8.csv' ,index_col=0)
train_df = train_df.drop(['delay_minutes','scheduled_time_departure','year'],axis=1)
x_train, y_train = train_df.drop('delayed',axis=1), train_df.loc[:,'delayed']
x_train.carrier = x_train.carrier.astype('category')
x_train.origin = x_train.origin.astype('category')
x_train.destination = x_train.destination.astype('category')
x_train.previous_is_delayed = x_train.previous_is_delayed.astype('category')

lgb_train = lgb.Dataset(x_train, y_train, categorical_feature = ['carrier','origin','destination','previous_is_delayed'], free_raw_data=False)

In [ ]:
dev_df = pd.read_csv('/Users/favea/Downloads/swiss-data/dev_df.csv', index_col=0)
dev_df = dev_df.drop(['Unnamed: 0','delay_minutes','scheduled_time_departure','year'],axis=1)

x_dev, y_dev = dev_df.drop('delayed',axis=1), dev_df.loc[:,'delayed']
x_dev.carrier = x_dev.carrier.astype('category')
x_dev.origin = x_dev.origin.astype('category')
x_dev.destination = x_dev.destination.astype('category')
x_dev.previous_is_delayed = x_dev.previous_is_delayed.astype('category')


lgb_dev = lgb.Dataset(x_dev, y_dev, reference=lgb_train, categorical_feature = ['carrier','origin','destination','previous_is_delayed'], free_raw_data=False)

2. Train baseline

In [ ]:
# specify your configurations as a dict
params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': ['binary_logloss','auc','average_precision'],
    'num_leaves': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbosity': 1,
    'scale_pos_weight': 1,
}

wandb.init(project='swiss-delay-prediction', job_type= 'training-lightgbm-baseline-balanced', config=params);

In [ ]:
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                valid_sets=lgb_dev,
                valid_names=('validation'),
                callbacks=[wandb_callback()],
                early_stopping_rounds=20,
                )

In [ ]:
log_summary(gbm, save_model_checkpoint=True)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, f1_score, precision_recall_fscore_support, roc_curve

In [ ]:
# predict
y_pred = gbm.predict(x_dev, num_iteration=gbm.best_iteration)

roc_ac = roc_auc_score(y_dev, y_pred)
wandb.log({'roc_auc': roc_ac})

In [ ]:
wandb.sklearn.plot_confusion_matrix(y_dev, (y_pred>=0.5).astype(int))

In [ ]:
y_train_pred = gbm.predict(x_train, num_iteration=gbm.best_iteration)
print(classification_report(y_dev, (y_pred>=0.5).astype(int)))

In [ ]:
d = classification_report(y_dev, (y_pred>=0.5).astype(int), output_dict=True)
sensitivity = d['1']['recall']
specificity = d['0']['recall']
f1_score = d['weighted avg']['f1-score']

wandb.summary['sensitivity'] = sensitivity
wandb.summary['specificity'] = specificity
wandb.summary['f1_score'] = f1_score

In [ ]:
false_positive_rate, true_positive_rate, thresholds = metrics.roc_curve(y_dev, y_pred) 

In [ ]:
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay
import matplotlib.pyplot as plt

In [ ]:
display = PrecisionRecallDisplay.from_predictions(    
    y_dev,
    y_pred,
    color="darkorange",
    name = 'LightGBM'
)
plt.plot()
wandb.log({'PR-curve':wandb.Image(plt)})

In [ ]:
RocCurveDisplay.from_predictions(
    y_dev,
    y_pred,
    color="darkorange",
)
plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
plt.axis("square")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend()
plt.show()

wandb.log({'ROC-curve':wandb.Image(plt)})

In [ ]:
wandb.finish()

## SWEEP

In [ ]:
sweep_config = {
  "method" : "bayes",
  "metric": {
      "name": "avg_precision_dev",
      "goal": "maximize"
  },
  "parameters" : {
    "learning_rate" :{
      "min": 0.001,
      "max": 1.0
    },
    "gamma" :{
      "min": 0.001,
      "max": 1.0
    },
    "min_child_weight" :{
      "min": 1,
      "max": 150
    },
    "early_stopping_rounds" :{
      "values" : [10, 30, 40]
    },
    "boosting_type":{"values":['gbdt','dart']},
    "num_leaves":{"values":[40,50, 60]},
    "feature_fraction": {"values":[0.4,0.9,1]},
    "bagging_fraction":{"values":[0.5,0.8,1]},
    "bagging_freq":{'values':[1,5,10]},
    "scale_pos_weight":{"values":[1, 0.8]},
    "num_iterations":{"values":[600,1000, 2000,3000]},
    "lambda_l1":{"values":[0.0, 0.3,0.5, 0.8]},
    "lambda_l2":{"values":[0.0, 0.3,0.5, 0.8]},

  }
}


In [ ]:
def train():     
    with wandb.init() as run:

        params = {
            'boosting_type': run.config['boosting_type'],
            'objective': 'binary',
            'metric': ['binary_logloss','auc','average_precision'],
            'num_leaves': run.config['num_leaves'],
            'learning_rate': run.config['learning_rate'],
            'feature_fraction': run.config['feature_fraction'],
            'bagging_fraction': run.config['bagging_fraction'],
            'bagging_freq': run.config['bagging_freq'],
            'verbosity': 1,
            'scale_pos_weight': run.config['scale_pos_weight'],
            'lambda_l2':run.config['lambda_l2'],
            'lambda_l1':run.config['lambda_l1'],

        }
    

        # Initialize and train LightGBM model
        gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=run.config['num_iterations'],
                        valid_sets=lgb_dev,
                        valid_names=('validation'),
                        callbacks=[wandb_callback()],
                        early_stopping_rounds=10,
                        )

        log_summary(gbm, save_model_checkpoint=True)
        


        # Log booster metrics
        run.summary["best_score"] = gbm.best_score
        run.summary["best_iteration"] = gbm.best_iteration
        
        # Get train and validation predictions
        y_dev_pred = gbm.predict(x_dev, num_iteration=gbm.best_iteration)
        y_train_pred = gbm.predict(x_train, num_iteration=gbm.best_iteration)


        # Log additional Train metrics
        false_positive_rate, true_positive_rate, thresholds = metrics.roc_curve(y_train, y_train_pred) 

        avg_precision_train = average_precision_score(y_train, y_train_pred)
        run.summary['train_avg_precision'] = avg_precision_train
        run.summary['train_ks_stat'] = max(true_positive_rate - false_positive_rate)
        run.summary['train_auc'] = metrics.auc(false_positive_rate, true_positive_rate)
        run.summary['train_log_loss'] = -(y_train * np.log(y_train_pred) + (1-y_train) * np.log(1-y_train_pred)).sum() / len(y_train)

        # Log additional Validation metrics
        avg_precision_dev = average_precision_score(y_dev, y_dev_pred)
        run.summary['avg_precision_dev'] = avg_precision_dev
        run.summary["val_auc"] = metrics.roc_auc_score(y_dev, y_dev_pred)
        run.summary["val_acc_0.5"] = metrics.accuracy_score(y_dev, np.where(y_dev_pred >= 0.5, 1, 0))
        run.summary["val_log_loss"] = -(y_dev * np.log(y_dev_pred) 
                                             + (1-y_dev) * np.log(1-y_dev_pred)).sum() / len(y_dev)

        d = classification_report(y_dev, np.where(y_dev_pred >= 0.5, 1, 0), output_dict=True)
        sensitivity = d['1']['recall']
        specificity = d['0']['recall']
        f1_score = d['weighted avg']['f1-score']

        run.summary['val_sensitivity_0.5'] = sensitivity
        run.summary['val_specificity_0.5'] = specificity
        run.summary['val_f1_score_0.5'] = f1_score


        # Log plots
        RocCurveDisplay.from_predictions(
            y_dev,
            y_dev_pred,
            color="darkorange",
        )
        plt.plot([0, 1], [0, 1], "k--", label="chance level (AUC = 0.5)")
        plt.axis("square")
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.legend()
        plt.show()

        run.log({'ROC-curve':wandb.Image(plt)})

        display = PrecisionRecallDisplay.from_predictions(    
            y_dev,
            y_dev_pred,
            color="darkorange",
            name = 'LightGBM'
        )
        plt.plot()
        plt.xlabel("Precision")
        plt.ylabel("Recall")
        plt.legend()
        run.log({'PR-curve':wandb.Image(plt)})


        #run.log({'feature_importance':fig})

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="swiss-delay-prediction")

In [ ]:
count = 50 # number of runs to execute
wandb.agent(sweep_id, function=train, count=count)

For heavily unbalanced datasets such as 1:10000:

max_bin: keep it only for memory pressure, not to tune (otherwise overfitting)

learning rate: keep it only for training speed, not to tune (otherwise overfitting)

n_estimators: must be infinite (like 9999999) and use early stopping to auto-tune (otherwise overfitting)

num_leaves: [7, 4095]

max_depth: [2, 63] and infinite (I personally saw metric performance increases with such 63 depth with small number of leaves on sparse unbalanced datasets)

scale_pos_weight: [1, 10000] (if over 10000, something might be wrong because I never saw it that good after 5000)

min_child_weight: [0.01, (sample size / 1000)] if you are using logloss (think about the hessian possible value rang
e before putting "sample size / 1000", it is dataset-dependent and loss-dependent)


subsample: [0.4, 1]


bagging_freq: only 1, keep as is (otherwise overfitting)

colsample_bytree: [0.4, 1]

is_unbalance: false (make your own weighting with scale_pos_weight)

USE A CUSTOM METRIC (to reflect reality without weighting, otherwise you have weights inside your metric with premade metrics like xgboost)

Never tune these parameters unless you have an explicit requirement to tune them:

Learning rate (lower means longer to train but more accurate, higher means smaller to train but less accurate)

Number of boosting iterations (automatically tuned with early stopping and learning rate)

Maximum number of bins (RAM dependent)